In [33]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

In [35]:
model.load_state_dict(torch.load('best_model.pth'))

FileNotFoundError: [Errno 2] No such file or directory: 'best_model.pth'

In [36]:
device = torch.device('cuda')
model = model.to(device)

AssertionError: Torch not compiled with CUDA enabled

In [37]:
import cv2 as cv
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [31]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked >= 0.5:
        print("blocked by obstacles")
    else:
        print("free to pass")
    
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to initialize

NameError: name 'camera' is not defined

NameError: name 'buzzer' is not defined